<a href="https://colab.research.google.com/github/Sagar-modelling/Machine_Learning_Totorials/blob/main/Keras_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 97 kB 3.1 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
tf.__version__

'2.6.0'

In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
train_images = train_images/255. #scale down
test_images = test_images/255.

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [11]:
tuner_search = RandomSearch(build_model, objective='val_accuracy',max_trials=5, directory = 'output', project_name ="mnist_fashion" )

In [13]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.3)

Trial 5 Complete [00h 01m 22s]
val_accuracy: 0.8748888969421387

Best val_accuracy So Far: 0.9018333554267883
Total elapsed time: 00h 04m 41s
INFO:tensorflow:Oracle triggered exit


In [15]:
model = tuner_search.get_best_models(num_models=1)[0]

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        13872     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1115184   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,130,378
Trainable params: 1,130,378
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1, initial_epoch=3) #retraining on best model

Epoch 4/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.2036 - accuracy: 0.9248 - val_loss: 0.2523 - val_accuracy: 0.9088
Epoch 5/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1566 - accuracy: 0.9416 - val_loss: 0.2726 - val_accuracy: 0.9072
Epoch 6/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1239 - accuracy: 0.9541 - val_loss: 0.2806 - val_accuracy: 0.9048
Epoch 7/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0963 - accuracy: 0.9647 - val_loss: 0.3001 - val_accuracy: 0.9090
Epoch 8/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0755 - accuracy: 0.9716 - val_loss: 0.3732 - val_accuracy: 0.9072
Epoch 9/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0564 - accuracy: 0.9794 - val_loss: 0.3927 - val_accuracy: 0.9092
Epoch 10/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0492 - accuracy: 0.9819 - val_loss: 0.4092 - val_a